# Get Data

In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import optimizers

In [2]:
NEpochs = 10000
BatchSize=250
#Optimizer=optimizers.SGD(lr=0.01)
Optimizer=optimizers.RMSprop(lr=0.001)

In [3]:
# Read in the data

TrainDF = pd.read_csv('/Users/hanxiao/Downloads/EmoryU/Fall/Machine Learning I/Homework/HM3_ML1/NNHWTrain.csv',sep=',',header=0,quotechar='"')
ValDF = pd.read_csv('/Users/hanxiao/Downloads/EmoryU/Fall/Machine Learning I/Homework/HM3_ML1/NNHWVal.csv',sep=',',header=0,quotechar='"')
TestDF = pd.read_csv('/Users/hanxiao/Downloads/EmoryU/Fall/Machine Learning I/Homework/HM3_ML1/NNHWTest.csv',sep=',',header=0,quotechar='"')

In [4]:
TrIsSpam = np.array(TrainDF['IsSpam'])

TrX = np.array(TrainDF.iloc[:,:-1])

## Rescale the training X data

In [5]:
TrXrsc = (TrX - TrX.min(axis=0))/TrX.ptp(axis=0)

In [6]:
# No need to rescale the Y because it is already 0 and 1. But check

print(TrIsSpam.min())
print(TrIsSpam.max())

0
1


## Rescale the validation data

In [7]:
ValIsSpam = np.array(ValDF['IsSpam'])

ValX = np.array(ValDF.iloc[:,:-1])

ValXrsc = (ValX - TrX.min(axis=0))/TrX.ptp(axis=0)

## Rescale the test data

In [8]:
TestIsSpam = np.array(TestDF['IsSpam'])

TestX = np.array(TestDF.iloc[:,:-1])

TestXrsc = (TestX - TrX.min(axis=0))/TrX.ptp(axis=0)

# Set up Neural Net Model

In [26]:
SpiralNN = Sequential()

SpiralNN.add(Dense(units=20,input_shape=(TrXrsc.shape[1],),activation="relu",use_bias=True))
SpiralNN.add(Dense(units=1,activation="sigmoid",use_bias=True))

SpiralNN.compile(loss='binary_crossentropy', optimizer=Optimizer,metrics=['binary_crossentropy','accuracy'])
print(SpiralNN.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 20)                1160      
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 21        
Total params: 1,181
Trainable params: 1,181
Non-trainable params: 0
_________________________________________________________________
None


# Fit NN Model

In [27]:
from keras.callbacks import EarlyStopping

StopRule = EarlyStopping(monitor='val_loss',mode='min',verbose=0,patience=100,min_delta=0.0)
FitHist = SpiralNN.fit(TrXrsc,TrIsSpam,validation_data=(ValXrsc,ValIsSpam), \
                    epochs=NEpochs,batch_size=BatchSize,verbose=0, \
                    callbacks=[StopRule])

In [28]:
print("Number of Epochs = "+str(len(FitHist.history['accuracy'])))
print("Final training accuracy: "+str(FitHist.history['accuracy'][-1]))
print("Recent history for training accuracy: "+str(FitHist.history['accuracy'][-10:-1]))
print("Final validation accuracy: "+str(FitHist.history['val_accuracy'][-1]))
print("Recent history for validation accuracy: "+str(FitHist.history['val_accuracy'][-10:-1]))

Number of Epochs = 1075
Final training accuracy: 0.9764578342437744
Recent history for training accuracy: [0.9757334589958191, 0.9764578342437744, 0.9764578342437744, 0.9764578342437744, 0.9757334589958191, 0.9760956168174744, 0.9771822094917297, 0.9768199920654297, 0.9771822094917297]
Final validation accuracy: 0.9413043260574341
Recent history for validation accuracy: [0.9434782862663269, 0.9434782862663269, 0.9423912763595581, 0.9413043260574341, 0.9402173757553101, 0.9423912763595581, 0.9423912763595581, 0.9423912763595581, 0.9413043260574341]


# Predictions

In [29]:
TrP = SpiralNN.predict(TrXrsc,batch_size=TrXrsc.shape[0])
ValP = SpiralNN.predict(ValXrsc,batch_size=ValXrsc.shape[0])
TestP = SpiralNN.predict(TestXrsc,batch_size=TestXrsc.shape[0])

In [30]:
TrainDF['TrP'] = TrP.reshape(-1)
ValDF['ValP'] = ValP.reshape(-1)
TestDF['TestP'] = TestP.reshape(-1)

In [31]:
TrainDF.to_csv('SpamNNWideTrainDFOutput.csv',sep=',',na_rep="NA",header=True,index=False)
ValDF.to_csv('SpamNNWideValDFOutput.csv',sep=',',na_rep="NA",header=True,index=False)
TestDF.to_csv('SpamNNWideTestDFOutput.csv',sep=',',na_rep="NA",header=True,index=False)

In [32]:
results = SpiralNN.evaluate(ValXrsc, ValIsSpam,
                                           batch_size=BatchSize, verbose=0)

for name, value in zip(SpiralNN.metrics_names, results):
  print(name, ': ', value)
print()

loss :  0.17839404940605164
binary_crossentropy :  0.17839404940605164
accuracy :  0.9413043260574341



In [33]:
dir(SpiralNN)

['_TF_MODULE_IGNORED_PROPERTIES',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_activity_regularizer',
 '_add_trackable',
 '_add_variable_with_custom_getter',
 '_assert_compile_was_called',
 '_assert_weights_created',
 '_auto_track_sub_layers',
 '_autocast',
 '_autographed_call',
 '_base_model_initialized',
 '_build_graph_network_for_inferred_shape',
 '_build_input_shape',
 '_call_accepts_kwargs',
 '_call_arg_was_passed',
 '_call_fn_arg_defaults',
 '_call_fn_arg_positions',
 '_call_fn_args',
 '_call_full_argspec',
 '_callable_losses',
 '_cast_single_input',
 '_check_call_args',
 '_checkpoint_dependencies',
 '_clear_los

# Narrow NN

In [35]:
SpiralNN = Sequential()

SpiralNN.add(Dense(units=4,input_shape=(TrXrsc.shape[1],),activation="relu",use_bias=True))
SpiralNN.add(Dense(units=4,activation="relu",use_bias=True))
SpiralNN.add(Dense(units=4,activation="relu",use_bias=True))
SpiralNN.add(Dense(units=4,activation="relu",use_bias=True))
SpiralNN.add(Dense(units=4,activation="relu",use_bias=True))
SpiralNN.add(Dense(units=1,activation="sigmoid",use_bias=True))

SpiralNN.compile(loss='binary_crossentropy', optimizer=Optimizer,metrics=['binary_crossentropy','accuracy'])
print(SpiralNN.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 4)                 232       
_________________________________________________________________
dense_11 (Dense)             (None, 4)                 20        
_________________________________________________________________
dense_12 (Dense)             (None, 4)                 20        
_________________________________________________________________
dense_13 (Dense)             (None, 4)                 20        
_________________________________________________________________
dense_14 (Dense)             (None, 4)                 20        
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 5         
Total params: 317
Trainable params: 317
Non-trainable params: 0
________________________________________________________

In [36]:
from keras.callbacks import EarlyStopping

StopRule = EarlyStopping(monitor='val_loss',mode='min',verbose=0,patience=100,min_delta=0.0)
FitHist = SpiralNN.fit(TrXrsc,TrIsSpam,validation_data=(ValXrsc,ValIsSpam), \
                    epochs=NEpochs,batch_size=BatchSize,verbose=0, \
                    callbacks=[StopRule])

In [37]:
print("Number of Epochs = "+str(len(FitHist.history['accuracy'])))
print("Final training accuracy: "+str(FitHist.history['accuracy'][-1]))
print("Recent history for training accuracy: "+str(FitHist.history['accuracy'][-10:-1]))
print("Final validation accuracy: "+str(FitHist.history['val_accuracy'][-1]))
print("Recent history for validation accuracy: "+str(FitHist.history['val_accuracy'][-10:-1]))

Number of Epochs = 658
Final training accuracy: 0.9637812376022339
Recent history for training accuracy: [0.9608837366104126, 0.9630568623542786, 0.9616081118583679, 0.9630568623542786, 0.9623324871063232, 0.9641434550285339, 0.9637812376022339, 0.9612459540367126, 0.9626947045326233]
Final validation accuracy: 0.9336956739425659
Recent history for validation accuracy: [0.935869574546814, 0.936956524848938, 0.935869574546814, 0.938043475151062, 0.935869574546814, 0.935869574546814, 0.936956524848938, 0.9347826242446899, 0.929347813129425]


In [38]:
TrP = SpiralNN.predict(TrXrsc,batch_size=TrXrsc.shape[0])
ValP = SpiralNN.predict(ValXrsc,batch_size=ValXrsc.shape[0])
TestP = SpiralNN.predict(TestXrsc,batch_size=TestXrsc.shape[0])

TrainDF['TrP'] = TrP.reshape(-1)
ValDF['ValP'] = ValP.reshape(-1)
TestDF['TestP'] = TestP.reshape(-1)

TrainDF.to_csv('SpamNNWideTrainDFOutput.csv',sep=',',na_rep="NA",header=True,index=False)
ValDF.to_csv('SpamNNWideValDFOutput.csv',sep=',',na_rep="NA",header=True,index=False)
TestDF.to_csv('SpamNNWideTestDFOutput.csv',sep=',',na_rep="NA",header=True,index=False)

In [39]:
results = SpiralNN.evaluate(ValXrsc, ValIsSpam,
                                           batch_size=BatchSize, verbose=0)

for name, value in zip(SpiralNN.metrics_names, results):
  print(name, ': ', value)
print()

loss :  0.20155967772006989
binary_crossentropy :  0.20155967772006989
accuracy :  0.9336956739425659

